In [1]:
import os
import torch
from torch import nn, optim, utils
import torch.nn.functional as F
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import swanlab

# CNN网络构建
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1 = nn.Conv2d(1, 10, 5)  # 10, 24x24
        self.conv2 = nn.Conv2d(10, 20, 3)  # 128, 10x10
        self.fc1 = nn.Linear(20 * 10 * 10, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        in_size = x.size(0)
        out = self.conv1(x)  # 24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  # 12
        out = self.conv2(out)  # 10
        out = F.relu(out)
        out = out.view(in_size, -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out, dim=1)
        return out


# 捕获并可视化前20张图像
def log_images(loader, num_images=16):
    images_logged = 0
    logged_images = []
    for images, labels in loader:
        # images: batch of images, labels: batch of labels
        for i in range(images.shape[0]):
            if images_logged < num_images:
                # 使用swanlab.Image将图像转换为wandb可视化格式
                logged_images.append(swanlab.Image(images[i], caption=f"Label: {labels[i]}"))
                images_logged += 1
            else:
                break
        if images_logged >= num_images:
            break
    swanlab.log({"MNIST-Preview": logged_images})
    

def train(model, device, train_dataloader, optimizer, criterion, epoch, num_epochs):
    model.train()
    # 1. 循环调用train_dataloader，每次取出1个batch_size的图像和标签
    for iter, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        # 2. 传入到resnet18模型中得到预测结果
        outputs = model(inputs)
        # 3. 将结果和标签传入损失函数中计算交叉熵损失
        loss = criterion(outputs, labels)
        # 4. 根据损失计算反向传播
        loss.backward()
        # 5. 优化器执行模型参数更新
        optimizer.step()
        print('Epoch [{}/{}], Iteration [{}/{}], Loss: {:.4f}'.format(epoch, num_epochs, iter + 1, len(train_dataloader),
                                                                      loss.item()))
        # 6. 每20次迭代，用SwanLab记录一下loss的变化
        if iter % 20 == 0:
            swanlab.log({"train/loss": loss.item()})

def test(model, device, val_dataloader, epoch):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        # 1. 循环调用val_dataloader，每次取出1个batch_size的图像和标签
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            # 2. 传入到resnet18模型中得到预测结果
            outputs = model(inputs)
            # 3. 获得预测的数字
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            # 4. 计算与标签一致的预测结果的数量
            correct += (predicted == labels).sum().item()
    
        # 5. 得到最终的测试准确率
        accuracy = correct / total
        # 6. 用SwanLab记录一下准确率的变化
        swanlab.log({"val/accuracy": accuracy}, step=epoch)
    

if __name__ == "__main__":

    #检测是否支持mps
    try:
        use_mps = torch.backends.mps.is_available()
    except AttributeError:
        use_mps = False

    #检测是否支持cuda
    if torch.cuda.is_available():
        device = "cuda"
    elif use_mps:
        device = "mps"
    else:
        device = "cpu"

    # 初始化swanlab
    run = swanlab.init(
        project="MNIST-example",
        experiment_name="PlainCNN",
        config={
            "model": "ResNet18",
            "optim": "Adam",
            "lr": 1e-4,
            "batch_size": 256,
            "num_epochs": 10,
            "device": device,
        },
    )

    # 设置MNIST训练集和验证集
    dataset = MNIST(os.getcwd(), train=True, download=True, transform=ToTensor())
    train_dataset, val_dataset = utils.data.random_split(dataset, [55000, 5000])

    train_dataloader = utils.data.DataLoader(train_dataset, batch_size=run.config.batch_size, shuffle=True)
    val_dataloader = utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)
    
    # （可选）看一下数据集的前16张图像
    log_images(train_dataloader, 16)

    # 初始化模型
    model = ConvNet()
    model.to(torch.device(device))

    # 打印模型
    print(model)

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=run.config.lr)

    # 开始训练和测试循环
    for epoch in range(1, run.config.num_epochs+1):
        swanlab.log({"train/epoch": epoch}, step=epoch)
        train(model, device, train_dataloader, optimizer, criterion, epoch, run.config.num_epochs)
        if epoch % 2 == 0: 
            test(model, device, val_dataloader, epoch)

    # 保存模型
    # 如果不存在checkpoint文件夹，则自动创建一个
    if not os.path.exists("checkpoint"):
        os.makedirs("checkpoint")
    torch.save(model.state_dict(), 'checkpoint/latest_checkpoint.pth')

/home/lick/tools/anaconda3/envs/VQ/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

swanlab: Tracking run with swanlab version 0.7.2

swanlab: Run data will be saved locally in 
/home/lick/project/VQ/callback/swanlog/run-20251203_140013-dwpepotb5rj1yjfsmwh1i

swanlab: 👋 Hi FreaKai,welcome to swanlab!

swanlab: Syncing run PlainCNN to the cloud

...

Epoch [8/10], Iteration [66/215], Loss: 0.0865
Epoch [8/10], Iteration [67/215], Loss: 0.0843
Epoch [8/10], Iteration [68/215], Loss: 0.1242
Epoch [8/10], Iteration [69/215], Loss: 0.1144
Epoch [8/10], Iteration [70/215], Loss: 0.1100
Epoch [8/10], Iteration [71/215], Loss: 0.1062
Epoch [8/10], Iteration [72/215], Loss: 0.1184
Epoch [8/10], Iteration [73/215], Loss: 0.0740
Epoch [8/10], Iteration [74/215], Loss: 0.0791
Epoch [8/10], Iteration [75/215], Loss: 0.0857
Epoch [8/10], Iteration [76/215], Loss: 0.1114
Epoch [8/10], Iteration [77/215], Loss: 0.1281
Epoch [8/10], Iteration [78/215], Loss: 0.1213
Epoch [8/10], Iteration [79/215], Loss: 0.0603
Epoch [8/10], Iteration [80/215], Loss: 0.1200
Epoch [8/10], Iteration [81/215], Loss: 0.1155
Epoch [8/10], Iteration [82/215], Loss: 0.0754
Epoch [8/10], Iteration [83/215], Loss: 0.0828
Epoch [8/10], Iteration [84/215], Loss: 0.1065
Epoch [8/10], Iteration [85/215], Loss: 0.1121
Epoch [8/10], Iteration [86/215], Loss: 0.0650
Epoch [8/10],

swanlab: Upload error: Connection was closed before we received a valid response from endpoint URL: 
"https://swanlab-beta-private-1301372061.cos.ap-nanjing.myqcloud.com/resources/tb5opmj3qtu8vy18eizzw/dwpepotb5rj1yj
fsmwh1i/MNIST-Preview/image-step0-c00a670395d3d9aa.png".

Epoch [9/10], Iteration [116/215], Loss: 0.1033
Epoch [9/10], Iteration [117/215], Loss: 0.1067
Epoch [9/10], Iteration [118/215], Loss: 0.0884
Epoch [9/10], Iteration [119/215], Loss: 0.1078
Epoch [9/10], Iteration [120/215], Loss: 0.1223
Epoch [9/10], Iteration [121/215], Loss: 0.0707
Epoch [9/10], Iteration [122/215], Loss: 0.0901
Epoch [9/10], Iteration [123/215], Loss: 0.1080
Epoch [9/10], Iteration [124/215], Loss: 0.0921
Epoch [9/10], Iteration [125/215], Loss: 0.1240
Epoch [9/10], Iteration [126/215], Loss: 0.0715
Epoch [9/10], Iteration [127/215], Loss: 0.0577
Epoch [9/10], Iteration [128/215], Loss: 0.1105
Epoch [9/10], Iteration [129/215], Loss: 0.0891
Epoch [9/10], Iteration [130/215], Loss: 0.0347
Epoch [9/10], Iteration [131/215], Loss: 0.0799
Epoch [9/10], Iteration [132/215], Loss: 0.0711
Epoch [9/10], Iteration [133/215], Loss: 0.0779
Epoch [9/10], Iteration [134/215], Loss: 0.1174
Epoch [9/10], Iteration [135/215], Loss: 0.0886
Epoch [9/10], Iteration [136/215], Loss:

swanlab: Upload error: Connection was closed before we received a valid response from endpoint URL: 
"https://swanlab-beta-private-1301372061.cos.ap-nanjing.myqcloud.com/resources/tb5opmj3qtu8vy18eizzw/dwpepotb5rj1yj
fsmwh1i/MNIST-Preview/image-step0-0394bc6215061281.png".

Epoch [10/10], Iteration [41/215], Loss: 0.1119
Epoch [10/10], Iteration [42/215], Loss: 0.0646
Epoch [10/10], Iteration [43/215], Loss: 0.0460
Epoch [10/10], Iteration [44/215], Loss: 0.0754
Epoch [10/10], Iteration [45/215], Loss: 0.0810
Epoch [10/10], Iteration [46/215], Loss: 0.0970
Epoch [10/10], Iteration [47/215], Loss: 0.0371
Epoch [10/10], Iteration [48/215], Loss: 0.0606
Epoch [10/10], Iteration [49/215], Loss: 0.0869
Epoch [10/10], Iteration [50/215], Loss: 0.0806
Epoch [10/10], Iteration [51/215], Loss: 0.0735
Epoch [10/10], Iteration [52/215], Loss: 0.0371
Epoch [10/10], Iteration [53/215], Loss: 0.0786
Epoch [10/10], Iteration [54/215], Loss: 0.0888
Epoch [10/10], Iteration [55/215], Loss: 0.1095
Epoch [10/10], Iteration [56/215], Loss: 0.0848
Epoch [10/10], Iteration [57/215], Loss: 0.0681
Epoch [10/10], Iteration [58/215], Loss: 0.0713
Epoch [10/10], Iteration [59/215], Loss: 0.0793
Epoch [10/10], Iteration [60/215], Loss: 0.0830
Epoch [10/10], Iteration [61/215], Loss: